In [1]:
from facebookads.adobjects.customaudience import CustomAudience
from facebookads.adobjects.adaccount import AdAccount
from facebookads.api import FacebookAdsApi
import json
import pandas as pd
import os
import datetime
import numpy as np
import glob
import time
import paramiko

BigLotsAccount_id="act_271491453638620"
BigLotsDcart10_id="act_2449979771956630"

list_account_id=[BigLotsAccount_id,BigLotsDcart10_id]
# jian_APP_ID="2537704939796694"# jian's fb marketing app id
jian_token=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/FB_token/lasest_FB_token_jian.json","r"))['latest_FB_token']


In [9]:
def recursive_file_gen(root_path):
    for root, dirs, files in os.walk(root_path):
        for file in files:
            yield os.path.join(root,file)
            
            
# create new adueince and add the new users to the audiecne, 1 step
# Please note the change of the CustomAudience.Schema (row 41 based on the user list format)
def creatfbaudience(AudienceName,
                    AudienceDescription,
                    df,
                    ACCESS_TOKEN,
                    accountid,
                    df_use_col_name,
                    fb_schema=CustomAudience.Schema.email_hash):
    FacebookAdsApi.init(access_token=ACCESS_TOKEN, api_version='v8.0')
    
    '''
    AudienceName: str, the name shown on FB
    AudienceDescription: str, the description shown on FB
    df: pd.Dataframe, input with the identifier col
    ACCESS_TOKEN: str, access to FB marketing api
    accountid: str, account to uplaod
    df_use_col_name: str, col name in the df
    fb_schema: available schema attribution in the CustomAudience.Schema class. Default for us CustomAudience.Schema.email_hash  
    https://github.com/facebook/facebook-python-business-sdk/blob/ac390147132101763a99a917bb59bdc8a374a7ce/facebook_business/adobjects/helpers/customaudiencemixin.py
    '''
    
    myAccount = AdAccount(accountid)
    fields = [
    ]
    params = {
      'name': AudienceName,
      'subtype': 'CUSTOM',
      'description': AudienceDescription,
      'customer_file_source': 'PARTNER_PROVIDED_ONLY',
    }
    fbname = myAccount.create_custom_audience(
      fields=fields,
      params=params,
    )['id']
    print("Audience ID:",fbname)
    audience = CustomAudience(fbname)
    users = list(df[df_use_col_name])
    listlen = len(users)
    chunck_size=50000
    count_uploaded=0
    for i in range(int(np.ceil(listlen/chunck_size))):
        starti = i*50000
        if (i+1)*50000<listlen:
            endi = (i+1)*50000
        else:
            endi = listlen
        list0 = users[starti:endi]
        
        count_uploaded+=len(list0)
        audience.add_users(fb_schema, list0)
        #print len(list0)   
    print(datetime.datetime.now(),AudienceName,"uploaded",count_uploaded)


In [3]:
# For recurrence
list_already_copied_deltas=list(recursive_file_gen("/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/PYD_2020Q4/files_from_client/"))
list_already_copied_deltas

[]

In [4]:
remote_data_path_client="/home/agilone/"

host = "107.191.32.220" #hard-coded
port = 22
transport = paramiko.Transport((host, port))

password = "Tur87ZqzF9xV5mYF" #hard-coded
username = "agilone" #hard-coded
transport.connect(username = username, password = password)
sftp = paramiko.SFTPClient.from_transport(transport)


In [5]:
list_remote_new_delta_file=sftp.listdir()

list_remote_new_delta_file=[x for x in list_remote_new_delta_file if "_PYD_" in x]
list_remote_new_delta_file

['12162020_PYD_Lapsed_MS_Hashed_20201215015235.csv',
 '12162020_PYD_SingUp_MS_Hashed_20201215015706.csv',
 '12162020_PYD_Active_MS_Hashed_20201215020409.csv']

In [6]:
date=set([x.split("_MS_Hashed_")[1][:8] for x in list_remote_new_delta_file])
if len(date)==1:
    date=list(date)[0]
else:
    raise ValueError("multiple dates")
print(date)

20201215


In [ ]:
# only copy new files
list_remote_new_delta_file=[x for x in list_remote_new_delta_file if os.path.basename(x) not in [os.path.basename(x) for x in list_already_copied_deltas]]

In [8]:
local_folder_new_files_delta="/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/PYD_2020Q4/files_from_client/File_%s/"%date
remote_movein_folder="/home/agilone/PYD_2020Q4/"

try:
    os.stat(local_folder_new_files_delta)
except:
    os.mkdir(local_folder_new_files_delta)


if len(list_remote_new_delta_file)==3:
    for file in list_remote_new_delta_file:
        local_file=local_folder_new_files_delta+os.path.basename(file)
        sftp.get(file,local_file)
        sftp.rename(file, remote_movein_folder+os.path.basename(file))      
else:
    raise ValueError("new received files not 3")

In [15]:
# Qc new file records
list_all_new_received=glob.glob(local_folder_new_files_delta+"*.csv")
for file in list_all_new_received:
    df=pd.read_csv(file,dtype=str)
    # row, unique id, unique email
    print(df.shape[0],df['customersummary_c_primaryscnhash'].nunique(),df['customersummary_emailhash'].nunique())

3567633 3567633 3567587
20120547 20120547 20098519
1879040 1879040 1879029


In [11]:
list_to_upload=glob.glob(local_folder_new_files_delta+"*.csv")
list_to_upload=[x for x in list_to_upload if "_PYD_Active_MS" in x] ## The note to upload filter per reqeust

list_to_upload.sort()
list_to_upload

['/mnt/clients/juba/hqjubaapp02/sharefolder/biglots_data/AgilOne/PYD_2020Q4/files_from_client/File_20201215/12162020_PYD_Active_MS_Hashed_20201215020409.csv']

In [16]:
col_identifer='customersummary_emailhash' # the col name in the new received dataframe
i=0
df_summary=pd.DataFrame()
for file in list_to_upload:
    for fb_account_id in list_account_id: # The 2 FB accounts
        i+=1
        audience_name = os.path.basename(file).split(".")[0]
        description=os.path.basename(file).split(".")[0]+"_uploaded_by_Juba_"+str(datetime.datetime.now().date())
        df = pd.read_csv(file,dtype=str) # load the dataframe contain email list 
        row_count=df.shape[0]
        df=df[df[col_identifer].apply(len)==64]
        email_unique_count=df[col_identifer].nunique() 
        
        df_summary_i=pd.DataFrame({'FB_account':fb_account_id,"audience_name":audience_name,"row_count":row_count,"unique_id":email_unique_count},index=[i])
        df_summary=df_summary.append(df_summary_i)
        print(audience_name, email_unique_count)
        creatfbaudience(AudienceName="aud_"+audience_name,
                        AudienceDescription=description+", file received from client on 20201215, 107 agile",
                        df=df,
                        df_use_col_name=col_identifer,
                        ACCESS_TOKEN=jian_token,
                        accountid=fb_account_id)

12162020_PYD_Active_MS_Hashed_20201215020409 20098518
Audience ID: 23846130655090265
2020-12-15 13:59:45.411105 aud_12162020_PYD_Active_MS_Hashed_20201215020409 uploaded 20100046
12162020_PYD_Active_MS_Hashed_20201215020409 20098518
Audience ID: 23846494421860632
2020-12-15 14:22:46.339806 aud_12162020_PYD_Active_MS_Hashed_20201215020409 uploaded 20100046


In [17]:
df_summary

,FB_account,audience_name,row_count,unique_id
1,act_271491453638620,12162020_PYD_Active_MS_Hashed_20201215020409,20120547,20098518
2,act_2449979771956630,12162020_PYD_Active_MS_Hashed_20201215020409,20120547,20098518
